# List of endpoints(404s) failing since last 5 days

In [ ]:
import pandas as pd
import numpy as np
import urllib
from datetime import date
import urllib.parse
import matplotlib 

datasette_url = "https://datasette.planning.data.gov.uk/"

params = urllib.parse.urlencode({
    "sql": f"""
     SELECT l.endpoint, l.status, l.exception, s.collection, s.organisation, l.entry_date
  FROM log l
  inner join source s
    on l.endpoint = s.endpoint where
   l.entry_date >= DATE('now', '-5 day')
   and l.status = '404'
   and s.collection = 'brownfield-land'
   GROUP BY l.endpoint
HAVING COUNT(DISTINCT l.entry_date) >= 5
order by s.organisation

    """,
    "_size": "max"
})



url = f"{datasette_url}digital-land.csv?{params}"
df = pd.read_csv(url)
print("\033[1m List of endpoints failing consecutively since more than 5 days")
df

# Organisation of Failed endpoints and finding out if that organisation has other latest endpoints

In [ ]:
#Organisation of Failed endpoints and finding out if that organisation has other latest endpoints
#organisations = ', '.join([f'"{organisation}"' for organisation in df['organisation']])

params = urllib.parse.urlencode({
    "sql": f"""
    SELECT s.collection, s.endpoint, s.end_date, s.organisation, s.source, MAX(s.entry_date) AS latest_entry_date, l.status, l.entry_date
FROM source s
INNER JOIN log l ON s.endpoint = l.endpoint
WHERE s.organisation IN (
    SELECT s2.organisation
    FROM log l2
    INNER JOIN source s2 ON l2.endpoint = s2.endpoint
    WHERE l2.entry_date >= DATE('now', '-5 day')
    AND l2.status = '404'
    AND s2.collection = 'brownfield-land'
    GROUP BY l2.endpoint
    HAVING COUNT(DISTINCT l2.entry_date) >= 5
)
AND s.collection = 'brownfield-land'
and s.end_date = '' and l.entry_date >= DATE('now', '-5 day')
GROUP BY s.organisation
    HAVING COUNT(DISTINCT l.entry_date) >= 5

    """,
    "_size": "max"
})

url = f"{datasette_url}digital-land.csv?{params}"
df = pd.read_csv(url)
print("\033[1m ")
df


In [ ]:
download = input("Do you want to download the result? (yes/no): ")

if download.lower() == "yes":
    # Save the DataFrame as a CSV file
    df.to_csv("query_result.csv", index=False)
    print("Query result downloaded as 'query_result.csv'")

In [ ]:
filtered_df = df[df['status'] == 404]
filtered_df

#local-authority-eng:CHS -- older date endpoint are passing and newly added endpoint is failing

In [ ]:
download = input("Do you want to download the result? (yes/no): ")

if download.lower() == "yes":
    # Save the DataFrame as a CSV file
    filtered_df.to_csv("endpoint_failing_result.csv", index=False)
    print("Query result downloaded as 'endpoint_failing_result.csv'")